### **TP Final TP Final Aprendizaje de Máquina I:**
Evaluación de la performance de modelos de clasificación con datos tumores de cáncer de mama en *malignos* o *benignos*.

Estudiantes:


*   Gustavo Julián Rivas
*   Myrna Lorena Degano
*   Santiago José Olaciregui
*   Fabricio Lopretto

## 4 - Algoritmos:

Originalmente la variable objetivo 'diagnosis' es categórica dado que puede tomar valors 'M' (maligno) o 'B' (benigno). Esto permite encarar la aplicación de algoritmos de clasificación. No obstante, la creación de la variable 'dummy' 'diagnosis_M' permite aplicar algoritmos de regresión dado la naturaleza numérica del objetivo creado.

A partir de lo anterior, se decidió aplicar y comparar los resultados de los siguientes modelos:

4.1 Regresión Lineal.

4.2 SVM en Regresión.

4.3 Árbol de Regresión.

4.4 Perceptrón.

4.5 Ensamble  por Boosting.

Los atributos categóricos son:

Atributo | Descripción | Unidad
:---: | :---: | :---:
`ID` | Número de registro. | adimensional
`diagnosis` | Clasificación en maligno o benigno. | adimensional           

Luego del análisis de datos y de la ingeniería de características, se obtuvo que las
características relevantes son 'radio', 'concavity' y 'concave_points'.
Además, se creó la variable 'dummy' 'diagnosis_M' que toma el valor de 1 cuando
el diagnóstico de tumor es maligno y 0 cuando es benigno.

Atributo | Descripción | Unidad
:---: | :---: | :---:
`radio` | Distancia media desde el centro a los puntos en el perímetro. | μm
`concavity` | Valor medio de la concavidad. | adimensional            
`concave_points` | Valor medio de número de porciones cóncavas del contorno. | adimensional        
`diagnosis_M` | Objetivo. Vale 1 con diagnóstico maligno. | adimensional


In [ ]:
"""
Este script permite implementar y evaluar el desempeño de modelos de regresión.
"""

In [1]:
# Importa las librerias nencesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Define la ruta al archivo de datos
directorio_actual = os.getcwd()
directorio_padre = os.path.dirname(directorio_actual)
directorio_datos = directorio_padre + '/Data/data_para_entrenamiento.xlsx'

In [3]:
# Carga del DataFrame con los registros que tienen los atributos y target.
df = pd.read_excel(directorio_datos, sheet_name='data')

In [4]:
# Visualiza informacion del conjunto de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   546 non-null    int64  
 1   diagnosis            546 non-null    object 
 2   radio_mean           546 non-null    float64
 3   concavity_mean       546 non-null    float64
 4   concave_points_mean  546 non-null    float64
 5   diagnosis_M          546 non-null    int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 25.7+ KB


In [5]:
# Importa split de datos, busqueda de hiper-parametros, estandarizacion y metricas
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score

In [6]:
# Separa las caracteristicas y el objetivo
X = df.loc[:, ["radio_mean", "concavity_mean", "concave_points_mean"]].values
y = df.loc[:, "diagnosis_M"].values

In [7]:
# Divide los datos en conjunto de entrenamiento y prueba 80/20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Normaliza el set de entrenamiento y de prueba de las caracteristicas
sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train) 
X_test = sc_X.transform(X_test)

#### 4.1 Regresión Lineal

In [9]:
# Importa el algoritmo
from sklearn.linear_model import LinearRegression

In [10]:
# Define el modelo de regresion lineal
model = LinearRegression()

In [11]:
# Define la grilla de hiper-parametros
param_grid = {
    'fit_intercept': [True, False],  # Si la ordenada al origen es nula o no
}

In [16]:
# Configura la busqueda de hiper-parametros por grilla con validacion cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error')

In [17]:
# Entrena el modelo
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False]},
             scoring='neg_mean_absolute_error')

In [18]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Mejores hiperparámetros: {'fit_intercept': True}


In [19]:
# Obtiene las metricas en el conjunto de prueba
y_pred = grid_search.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"El Error Absoluto Medio (MAE) en el conjunto de prueba: {mae}")
print(f"El Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse}")
print(f"La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse}")
print(f"El Coeficiente de Ajuste (R2) en el conjunto de prueba: {r2}")

El Error Absoluto Medio (MAE) en el conjunto de prueba: 0.24712953623628392
El Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.10320129531188953
La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: 0.32124958414274957
El Coeficiente de Ajuste (R2) en el conjunto de prueba: 0.5803979592493741


#### 4.2 SVM en Regresión

In [20]:
# Importa el algoritmo
from sklearn.svm import SVR

In [21]:
# Define el modelo de SVM de regresion
model = SVR()

In [22]:
# Define la grilla de hiper-parametros
grid_search = GridSearchCV(model,
                    [{"C": [0.01, 0.1, 1, 5, 10, 100], "kernel": ["linear"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "gamma": [0.1, 0.5, 1, 2, 10, 100], "kernel": ["rbf", "sigmoid"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "degree": [2, 3, 4, 5, 6], "kernel": ["poly"]}],
                    refit=True,
                    cv=5,
                    scoring='neg_mean_absolute_error') # MAE es mejor en el caso de regresion


In [23]:
# Entrena el modelo
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.01, 0.1, 1, 5, 10, 100],
                          'kernel': ['linear']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'gamma': [0.1, 0.5, 1, 2, 10, 100],
                          'kernel': ['rbf', 'sigmoid']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'degree': [2, 3, 4, 5, 6], 'kernel': ['poly']}],
             scoring='neg_mean_absolute_error')

In [24]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Mejores hiperparámetros: {'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}


In [25]:
# Obtiene las metricas en el conjunto de prueba
svm_best = grid_search.best_estimator_
y_pred = svm_best.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"El Error Absoluto Medio (MAE) en el conjunto de prueba: {mae}")
print(f"El Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse}")
print(f"La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse}")
print(f"El Coeficiente de Ajuste (R2) en el conjunto de prueba: {r2}")

El Error Absoluto Medio (MAE) en el conjunto de prueba: 0.1667901570988707
El Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.09146670722812886
La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: 0.3024346329839373
El Coeficiente de Ajuste (R2) en el conjunto de prueba: 0.6281091540791806


#### 4.3 Árbol de Regresión

In [26]:
# Importa el algoritmo
from sklearn.tree import DecisionTreeRegressor

In [27]:
# Crea el modelo de arbol de regresion
model = DecisionTreeRegressor()

In [28]:
# Define la grilla de hiper-parametros
param_grid = [
    {"criterion": ["squared_error"],
     "splitter": ["best"],
     "max_depth": [x for x in range(10)],
     "min_samples_split": [x for x in range(20)],
     "min_samples_leaf": [x for x in range(10)],
     "max_features": [None],
     "ccp_alpha": [x * 0.01 for x in range(0, 50)]}
]

In [29]:
# Configura GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error', refit=True)

In [30]:
# Entrena el modelo
grid_search.fit(X_train, y_train)

c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
135500 fits failed out of a total of 500000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50000 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\skle

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'ccp_alpha': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                        0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13,
                                        0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2,
                                        0.21, 0.22, 0.23, 0.24, 0.25, 0.26,
                                        0.27, 0.28, 0.29, ...],
                          'criterion': ['squared_error'],
                          'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'max_features': [None],
                          'min_samples_leaf': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'min_samples_split': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                10, 11, 12, 13, 14, 15, 16, 17,
                                                18, 19],
                          'splitter': ['best']}],
             scoring='neg_mean_absolute_error')

In [31]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Mejores hiperparámetros: {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 6, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 7, 'splitter': 'best'}


In [32]:
# Usar el mejor modelo para predecir
best_model = grid_search.best_estimator_

In [33]:
# Obtiene las metricas en el conjunto de prueba
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"El Error Absoluto Medio (MAE) en el conjunto de prueba: {mae}")
print(f"El Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse}")
print(f"La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse}")
print(f"El Coeficiente de Ajuste (R2) en el conjunto de prueba: {r2}")

El Error Absoluto Medio (MAE) en el conjunto de prueba: 0.13406204906204905
El Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.10104381971643873
La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: 0.3178739053719867
El Coeficiente de Ajuste (R2) en el conjunto de prueba: 0.5891699534378668


#### 4.4 Perceptrón

In [34]:
# Importa el algoritmo
from sklearn.linear_model import Perceptron

In [38]:
# Crea el modelo de perceptron
model = Perceptron()

In [39]:
# Define la grilla de hiper-parametros
param_grid = {
    'penalty': [None, 'l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4, 1e-5]
}

In [40]:
# Realiza la busqueda de hiperparametros con GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')

In [41]:
# Entrena el modelo
grid_search.fit(X_train, y_train)

c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'max_iter': [1000, 2000, 3000],
                         'penalty': [None, 'l2', 'l1', 'elasticnet'],
                         'tol': [0.001, 0.0001, 1e-05]},
             scoring='neg_mean_absolute_error')

In [42]:
# Obtiene los mejores hiperparametros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'alpha': 0.001, 'max_iter': 1000, 'penalty': 'l2', 'tol': 0.001}


In [43]:
# Entrena el modelo con los mejores hiperparametros
best_model = grid_search.best_estimator_

In [44]:
# Obtiene las metricas en el conjunto de prueba
y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"El Error Absoluto Medio (MAE) en el conjunto de prueba: {mae}")
print(f"El Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse}")
print(f"La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse}")
print(f"El Coeficiente de Ajuste (R2) en el conjunto de prueba: {r2}")

El Error Absoluto Medio (MAE) en el conjunto de prueba: 0.13636363636363635
El Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.13636363636363635
La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: 0.3692744729379982
El Coeficiente de Ajuste (R2) en el conjunto de prueba: 0.4455645161290325


#### 4.5 Ensamble por Boosting

In [45]:
# Importa el algoritmo
from xgboost import XGBRegressor

In [46]:
# Crea el ensamble
model = XGBRegressor()

In [49]:
# Define los hiperparametros a buscar
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [50]:
# Realiza la busqueda de hiperparametros con GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')

In [51]:
# Entrena el modelo
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='neg_mean_absolute_error')

In [52]:
# Obtiene los mejores hiperparametros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}


In [53]:
# Entrena el modelo con los mejores hiperparametros
best_model = grid_search.best_estimator_

In [54]:
# Realiza predicciones sobre el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcula metricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"El Error Absoluto Medio (MAE) en el conjunto de prueba: {mae}")
print(f"El Error Cuadrático Medio (MSE) en el conjunto de prueba: {mse}")
print(f"La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: {rmse}")
print(f"El Coeficiente de Ajuste (R2) en el conjunto de prueba: {r2}")

El Error Absoluto Medio (MAE) en el conjunto de prueba: 0.043412238359451294
El Error Cuadrático Medio (MSE) en el conjunto de prueba: 0.007043506950139999
La Raíz Error Cuadrático Medio (RMSE) en el conjunto de prueba: 0.08392560482025146
El Coeficiente de Ajuste (R2) en el conjunto de prueba: 0.9638357162475586


Se realizó una tabla comparativa de los resultados de las métricas sobre el conjunto datos de prueba:

| Modelo             |  MAE  |  MSE  |  RMSE |  R2   |
|--------------------|-------|-------|-------|-------|
| Regresión Lineal   | 0.247 | 0.103 | 0.321 | 0.583 |
| SVM de Regresión   | 0.167 | 0.091 | 0.302 | 0.628 |
| Árbol de Regresión | 0.134 | 0.101 | 0.318 | 0.589 |
| Perceptrón         | 0.136 | 0.136 | 0.369 | 0.446 |
| Ensamble Boosting  | 0.043 | 0.007 | 0.084 | 0.964 |

Conclusiones: Considerando el MAE los mejores resultados fueron el árbol de regresión y el perceptrón, mientras que usando RMSE es SVM de regresión. Por otro lado, considerando el coeficiente de ajuste y el MSE el ensamble fue el que mejor resultados dio. 

Bibliografía:

Isabelle Guyon, Andre Elisseeff (2003). An introduction to variable and feature selection. Journal of Machine Learning Research 3 (2003) 1157-1182


Wolberg,William, Mangasarian,Olvi, Street,Nick, and Street,W.. (1995). Breast Cancer Wisconsin (Diagnostic). UCI Machine Learning Repository. https://doi.org/10.24432/C5DW2B.


W. Street, W. Wolberg, O. Mangasarian. (1992)
Nuclear feature extraction for breast tumor diagnosis.
Published in Electronic imaging.


